# Data Cleaning and Transforming
Data source: https://www.datafiles.samhsa.gov/dataset/nsduh-2002-2019-ds0001-nsduh-2002-2019-ds0001 

### Column Transformation: amt_paid, who_paid_outpat, who_paid_inpat
    - Input: newdata.csv 
    - Output: newdata_v2.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('newdata')
# Strip whitespaces
df = df.apply(lambda x:x.str.strip() if x.dtype == "object" else x, axis = 0)

# Drop underage records
l = []
for item in df.age.unique():
    l.append(item)
underaged_index = [8,11,12,14,15,16]
underage = []
for i in underaged_index:
    underage.append(l[i])
df = df[~df['age'].isin(underage)]

In [3]:
# transform amt_paid_inpat column
df['amt_paid_inpat'] = df['amt_paid_inpat'].map({1: 'Less than $100',
                                                2: '$100 to $200',
                                                3: '$201 to $500',
                                                4: '$501 to $900',
                                                5: '$901 to $1,500',
                                                6: '1,501 to $2,000',
                                                7: '$2,001 to $5,000',
                                                8: 'More than $5,000',
                                                85: 'bad data',
                                                94: 'do not know',
                                                97: 'refused',
                                                98: 'blank',
                                                99: 'skip'})
# check to make sure the transformation worked
df['amt_paid_inpat'].value_counts()

skip                597213
refused                549
blank                  514
Less than $100         219
More than $5,000       187
do not know            116
$100 to $200           116
$2,001 to $5,000       109
$901 to $1,500          99
$201 to $500            96
$501 to $900            89
1,501 to $2,000         76
bad data                24
Name: amt_paid_inpat, dtype: int64

In [4]:
# transform amt_paid_outpat column
df['amt_paid_outpat'] = df['amt_paid_outpat'].map({1: 'Less than $100',
                                                2: '$100 to $200',
                                                3: '$201 to $500',
                                                4: '$501 to $900',
                                                5: '$901 to $1,500',
                                                6: '1,501 to $2,000',
                                                7: '$2,001 to $5,000',
                                                8: 'More than $5,000',
                                                85: 'bad data',
                                                94: 'do not know',
                                                97: 'refused',
                                                98: 'blank',
                                                99: 'skip'})

# check to make sure the transformation worked
df['amt_paid_outpat'].value_counts()

skip                587254
Less than $100        3071
$201 to $500          1976
$100 to $200          1963
$501 to $900          1095
blank                  985
refused                764
$901 to $1,500         659
do not know            600
$2,001 to $5,000       421
1,501 to $2,000        378
More than $5,000       218
bad data                23
Name: amt_paid_outpat, dtype: int64

In [5]:
# transform fam_paid_inpat
df['fam_paid_inpat'] = df['fam_paid_inpat'].map({1: 'Self or live-in family member'})
# transform fam_paid_inpat_away
df['fam_paid_inpat_away'] = df['fam_paid_inpat_away'].map({1: 'Family member not living with you'})
# transform private_ins_paid_inpat
df['private_ins_paid_inpat'] = df['private_ins_paid_inpat'].map({1: 'Private insurance'})
# transform medicare_paid_inpat
df['medicare_paid_inpat'] = df['medicare_paid_inpat'].map({1: 'Medicare'})
# transform medicaid_paid_inpat
df['medicaid_paid_inpat'] = df['medicaid_paid_inpat'].map({1: 'Medicaid'})
# transform employer_paid_inpat
df['employer_paid_inpat'] = df['employer_paid_inpat'].map({1: 'Employer'})
# transform military_paid_inpat
df['military_paid_inpat'] = df['military_paid_inpat'].map({1: 'Military'})
# transform other_pub_inpat
df['other_pub_inpat'] = df['other_pub_inpat'].map({1: 'Other public source'})
# transform other_private_inpat
df['other_private_inpat'] = df['other_private_inpat'].map({1: 'Other private source'})
# transform care_free
df['care_free'] = df['care_free'].map({1: 'Care was free'})

# check to make sure the transformation worked
df['care_free'].value_counts()

Care was free    313
Name: care_free, dtype: int64

In [6]:
# create new column called amt_paid to combine amt_paid_inpat and amt_paid_outpat
df.loc[df['rcvd_inpat'] == 1, 'amt_paid'] = df['amt_paid_inpat']
df.loc[df['rcvd_outpat'] == 1, 'amt_paid'] = df['amt_paid_outpat']

In [7]:
# merge columns together
df2 = pd.concat([df['ID'], df['fam_paid_inpat']
                .combine_first(df['fam_paid_inpat_away'])
               .combine_first(df['private_ins_paid_inpat'])
               .combine_first(df['medicare_paid_inpat'])
               .combine_first(df['medicaid_paid_inpat'])
               .combine_first(df['employer_paid_inpat'])
               .combine_first(df['military_paid_inpat'])
               .combine_first(df['other_pub_inpat'])
               .combine_first(df['other_private_inpat'])
               .combine_first(df['care_free'])], axis = 1)

df2 = df2.rename(columns = {'fam_paid_inpat':'who_paid_inpat'})
df2 = df2.rename(columns = {'ID':'ID2'})

In [8]:
df2['who_paid_inpat'].value_counts()

Self or live-in family member        987
Medicaid                             696
Medicare                             589
Private insurance                    517
Care was free                        297
Family member not living with you    158
Other public source                  119
Military                             110
Employer                              96
Other private source                  44
Name: who_paid_inpat, dtype: int64

In [9]:
# combine new column with original df; store to a new csv file
df = pd.concat([df, df2], join = 'outer', axis = 1)
df = df.drop('ID2', axis = 1)
final_cols = ['ID',
'year',
'age',
'sex',
'race',
'fam_income',
'medicare',
'caidchip',
'champus',
'prvhltin',
'grphltin',
'rcvd_inpat',
'rcvd_outpat',
'rcvd_rx',
'MH_covered',
'unmet_need',
'no_care_reasons',
'amt_paid_inpat',
'who_paid_inpat',
'who_paid_outpat',
'amt_paid']
df = df[final_cols]
df.to_csv('newdata_v2')

### Column Transformation: fam_income, MH_covered, unmet_need, no_care_reasons
    - Input: newdata_v2.csv
    - Output: newdata_v3.csv

In [10]:
#Transform frome code from 1-7 to actual incomes of families.
df['fam_income'] = df['fam_income'].map({1: 'Less than $10,000 (including loss)',
                                                2: '$10,000 to $19,999',
                                                3: '$20,000 to $29,999',
                                                4: '$30,000 to $39,999',
                                                5: '$40,000 to $49,999',
                                                6: '$50,000 to $74,999',
                                                7: '$75,000 or more'})
df['fam_income'].value_counts()

$75,000 or more                       156931
$50,000 to $74,999                     98015
$10,000 to $19,999                     78353
$20,000 to $29,999                     72838
$30,000 to $39,999                     67357
$40,000 to $49,999                     64948
Less than $10,000 (including loss)     60965
Name: fam_income, dtype: int64

In [11]:
#Transform from code to an answer a user could understand.
df['MH_covered'] = df['MH_covered'].map({1: 'Yes',
                                                2: 'No',
                                                94: 'Do not Know',
                                                97: 'Refused',
                                                98: 'Blank (No Answer)',
                                                99: 'LEGITIMATE SKIP (PRVHLTIN = 2)'})
df['MH_covered'].value_counts()

Yes                               237254
LEGITIMATE SKIP (PRVHLTIN = 2)    233493
Do not Know                        93811
No                                 30283
Blank (No Answer)                   4076
Refused                              477
Name: MH_covered, dtype: int64

In [12]:
#Transform code to an actual answer a user could understand. 
df['unmet_need'] = df['unmet_need'].map({1: 'Yes',
                                                2: 'No',
                                                85: 'Bad Data',
                                                94: 'Do not Know',
                                                97: 'Refused',
                                                98: 'Blank (No Answer)',
                                                99: 'LEGITIMATE SKIP'})
df['unmet_need'].value_counts()

No                   323127
LEGITIMATE SKIP      247125
Yes                   27796
Refused                 651
Do not Know             587
Blank (No Answer)        69
Bad Data                 43
Name: unmet_need, dtype: int64

In [13]:
#Transform code to an actual reason of no care instead of answer being a number.
df['no_care_reasons'] = df['no_care_reasons'].map({1: 'Could \nnot afford\n the cost',
                                                2: 'Concerned neighbors/\ncommunity have neg opinion',
                                                3: 'Concerned might have a neg effect on your job',
                                                4: 'Health ins doesnt cover any mental hlth tmt',
                                                5: 'Health ins doesnt pay enuf for ment hlth tmt',
                                                6: 'Did not know where to go to get services',
                                                7: 'Concerned info might not be kept confidential',
                                                8: 'Concerned might be committed/take medicine .',
                                                9: 'Did not think \n you needed \n treatment\n at the time',
                                                10: 'Thought \ncould handle\n problem w/o\n treatment ',
                                                11: 'Did not think treatment would help',
                                                12: 'Did not have\n time (b/c \nof job, \nchildcare, \nother)',
                                                13: 'Did not\n want others\n to find out \nyou needed\n treatmnt',
                                                14: 'No transportation/too far away/hrs inconvenient',
                                                15: 'Some other reason or reasons',
                                                16: 'Afraid too painful/unpleasant/afraid of diagnosis',
                                                17: 'Others \ndissuaded/did\n not want \nyou to/\nunsupportive',
                                                18: 'No health \n insurance\n coverage',
                                                19: 'Ashamed, \nembarrassed,\n afraid, \nor sign of \nweakness',
                                                21: 'Did not want to; reason unspecified',
                                                22: 'Unmotivated/\ndepressed\n/confused\n/angry\n/unworthy',
                                                23: 'Services unavailable in your area/services limited',
                                                24: 'Did not want to talk about your problems w/anyone',
                                                25: 'Did not\n like how \ntreated/do \nnot like \nDr(s)/hospitals',
                                                26: 'Attempted to\n get trmt,\nunsuccessful in\n finding help',
                                                27: 'Concerned how the court system would treat you',
                                                28: 'No program/counselor competent/comfortable with',
                                                29: 'Just never went/made appointment/arrangements',
                                                30: 'Work on your problems with family/friends',
                                                32: 'Too much red tape/hassle to get services',
                                                33: 'No openings/\nlong waiting\n list/delays',
                                                34: 'Had other problems/issues to deal with',
                                                37: 'Were using alcohol/drugs',
                                                38: 'Too afraid to leave the house',
                                                41: 'Preferrd providr wont/may not accept hlth ins plan',
                                                42: 'Health ins wont accept preferred provider/tmt',
                                                44: 'Services desired unavailable/currently ineligible',
                                                70: 'Did/had appt for mental health trmt/counseling',
                                                85: 'Bad Data',
                                                94: 'Do not Know',
                                                97: 'Refused',
                                                98: 'Blank (No Answer)',
                                                99: 'LEGITIMATE SKIP'})
df['no_care_reasons'].value_counts()

LEGITIMATE SKIP                                                 593340
Refused                                                            925
Blank (No Answer)                                                  830
Did not think \n you needed \n treatment\n at the time             286
Ashamed, \nembarrassed,\n afraid, \nor sign of \nweakness          279
No health \n insurance\n coverage                                  263
Could \nnot afford\n the cost                                      248
Did not have\n time (b/c \nof job, \nchildcare, \nother)           244
Unmotivated/\ndepressed\n/confused\n/angry\n/unworthy              230
No openings/\nlong waiting\n list/delays                           194
Thought \ncould handle\n problem w/o\n treatment                   191
Did not\n want others\n to find out \nyou needed\n treatmnt        172
Did not\n like how \ntreated/do \nnot like \nDr(s)/hospitals       154
Others \ndissuaded/did\n not want \nyou to/\nunsupportive          152
Attemp

In [14]:
df.to_csv('newdata_v3.csv')

### Column Transformation: sex, race, hltsrv, hltin
    - Input: rawdata.csv
    - Output: sex_race_hltin_hltsrv.csv to be merged to newdata_v3.csv

In [15]:
df = pd.read_csv('newdata_v3.csv')

# Strip whitespaces
df = df.apply(lambda x:x.str.strip() if x.dtype == "object" else x, axis = 0)

# sex column transformation
ini = [item for item in df.sex.unique()]
genders = ['Male', 'Female']
df['sex'] = df['sex'].replace(ini, genders)

# race column transformation
ini = [item for item in df.race.unique()]
races = ['White', 'Asian', 'Black/African American', 'Multiple Races', 'Hispanic', 'Native Highlander/Other Pacific Islander', 'American Indian/Alaskan Natives']
df['race'] = df['race'].replace(ini, races)

In [16]:
# check transformation
df.sex.value_counts()

Female    308384
Male      291023
Name: sex, dtype: int64

In [17]:
# check transformation
df.race.value_counts()

White                                       360840
Multiple Races                              104047
Hispanic                                     78812
Asian                                        22321
Black/African American                       21384
Native Highlander/Other Pacific Islander      8989
American Indian/Alaskan Natives               3014
Name: race, dtype: int64

In [18]:
# inpatient/outpatient/rx column combination into hlt_srv
df['rcvd_inpat'] = df['rcvd_inpat'].map({
    1 :'Yes',
    2: 'No',
    np.nan: 'Unknown'
})

df['rcvd_outpat'] = df['rcvd_outpat'].map({
    1:'Yes',
    2: 'No',
    np.nan: 'Unknown'
})

df['rcvd_rx'] = df['rcvd_rx'].map({
    1:'Yes',
    2: 'No',
    np.nan: 'Unknown'
})

# Combine columns in Dataframe test to column hlt_srv
test = df.copy()
mask1 = (test['rcvd_inpat'] != 'Yes') & (test['rcvd_outpat'] != 'Yes') & (test['rcvd_rx'] != 'Yes') & (~(test['rcvd_inpat'] == 'No') & (test['rcvd_outpat'] == 'No') & (test['rcvd_rx'] == 'No'))
test.loc[mask1,'hlt_sv'] = 'Unknown'
mask2 = (test['rcvd_inpat'] == 'No') & (test['rcvd_outpat'] == 'No') & (test['rcvd_rx'] == 'No')
test.loc[mask2,'hlt_sv'] = 'None'
mask3 = (test['rcvd_inpat'] == 'Yes') & (test['rcvd_outpat'] != 'Yes') & (test['rcvd_rx'] != 'Yes')
test.loc[mask3,'hlt_sv'] = 'Inpatient'
mask4 = (test['rcvd_inpat'] != 'Yes') & (test['rcvd_outpat'] == 'Yes') & (test['rcvd_rx'] != 'Yes')
test.loc[mask4,'hlt_sv'] = 'Outpatient'
mask5 = (test['rcvd_inpat'] != 'Yes') & (test['rcvd_outpat'] != 'Yes') & (test['rcvd_rx'] == 'Yes')
test.loc[mask5,'hlt_sv'] = 'Prescriptions'
mask6 = (test['rcvd_inpat'] == 'Yes') & (test['rcvd_outpat'] == 'Yes') & (test['rcvd_rx'] != 'Yes')
test.loc[mask6,'hlt_sv'] = 'Inpatient and Outpatient'
mask7 = (test['rcvd_inpat'] == 'Yes') & (test['rcvd_outpat'] != 'Yes') & (test['rcvd_rx'] == 'Yes')
test.loc[mask7,'hlt_sv'] = 'Inpatient and Prescriptions'
mask8 = (test['rcvd_inpat'] != 'Yes') & (test['rcvd_outpat'] == 'Yes') & (test['rcvd_rx'] == 'Yes')
test.loc[mask8,'hlt_sv'] = 'Outpatient and Prescriptions'
mask9 = (test['rcvd_inpat'] == 'Yes') & (test['rcvd_outpat'] == 'Yes') & (test['rcvd_rx'] == 'Yes')
test.loc[mask9,'hlt_sv'] = 'Inpatient, Outpatient and Prescriptions'

# Add new column hltsrv to df base on hlt_srv from test
df.loc[:,'hltsrv'] = test['hlt_sv']
df.hltsrv.value_counts()

None                                       302660
Prescriptions                               20707
Outpatient and Prescriptions                15055
Outpatient                                   8382
Inpatient, Outpatient and Prescriptions      1789
Inpatient                                    1022
Inpatient and Prescriptions                   649
Inpatient and Outpatient                      389
Unknown                                        66
Name: hltsrv, dtype: int64

In [19]:
# Types of Insurance: combine into hltin

# Make a copy of df and select useable columns
test = df.copy()
test = test[['medicare', 'caidchip', 'champus', 'prvhltin', 'grphltin']]
for col in ['medicare', 'caidchip', 'champus', 'prvhltin', 'grphltin']:
    mask = ~test[col].isin([1,2])
    test.loc[mask,col] = 'Other'

# Combine group and private insurances
mask = test['prvhltin'] != 1
test.loc[mask, 'prv_grp_comb'] = 'No'
mask = (test['prvhltin'] == 1) & (test['grphltin'] != 1)
test.loc[mask, 'prv_grp_comb'] = 'prv'
mask = (test['prvhltin'] == 1) & (test['grphltin'] == 1)
test.loc[mask, 'prv_grp_comb'] = 'grp'

# Combine all the insurances into one column
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicare, Medicaid, Military, and Private Health Insurance'
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicare, Medicaid, Military, and Group Offered Health Insurance'
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicare, Medicaid, and Military'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicare, Military, and Private Health Insurance'
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicare, Medicaid, and Private Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicaid, Military, and Private Health Insurance'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicare, Military, and Group Offered Health Insurance'
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicare, Medicaid, and Group Offered Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicaid, Military, and Group Offered Health Insurance'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicare and Private Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Medicaid and Private Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Military and Private Health Insurance'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicare and Group Offered Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Medicaid and Group Offered Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Military and Group Offered Health Insurance'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicare and Military'
mask = (test['medicare'] == 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicare and Medicaid'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicaid and Military'

mask = (test['medicare'] == 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicare'
mask = (test['medicare'] != 1) & (test['caidchip'] == 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Medicaid'
mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] == 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'Military'

mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'prv')
test.loc[mask, 'hltin'] = 'Private Health Insurance'
mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'grp')
test.loc[mask, 'hltin'] = 'Group Offered Health Insurance'

mask = (test['medicare'] != 1) & (test['caidchip'] != 1) & (test['champus'] != 1) & (test['prv_grp_comb'] == 'No')
test.loc[mask, 'hltin'] = 'None'

# Add the new column to the df
df.loc[:,'hltin'] = test['hltin']

In [20]:
# Check transformation
df.hltin.value_counts()

Group Offered Health Insurance                                      314227
Medicaid                                                            110296
None                                                                106464
Private Health Insurance                                             28264
Military                                                             12317
Medicaid and Group Offered Health Insurance                          10151
Military and Group Offered Health Insurance                           4220
Medicare and Medicaid                                                 4061
Medicare                                                              3163
Medicare and Group Offered Health Insurance                           1502
Medicaid and Private Health Insurance                                 1311
Medicaid and Military                                                 1097
Medicare and Private Health Insurance                                  625
Medicare and Military    

In [21]:
# Remove columns; convert to csv
final_cols = ['ID',
'year',
'age',
'sex',
'race',
'fam_income',
'hltin',
'hltsrv',
'MH_covered',
'unmet_need',
'no_care_reasons',
'amt_paid_inpat',
'who_paid_inpat',
'who_paid_outpat',
'amt_paid']
df = df[final_cols]
df.to_csv('final_data')

In [22]:
pd.read_csv('final_data')

,Unnamed: 0,ID,year,age,sex,race,fam_income,hltin,hltsrv,MH_covered,unmet_need,no_care_reasons,amt_paid_inpat,who_paid_inpat,who_paid_outpat,amt_paid
0,0,2000024,2002,13,Male,White,"$50,000 to $74,999",Group Offered Health Insurance,None,Yes,No,LEGITIMATE SKIP,skip,NaN,99,NaN
1,1,2000239,2002,12,Female,Asian,"$75,000 or more",Group Offered Health Insurance,None,Do not Know,No,LEGITIMATE SKIP,skip,NaN,99,NaN
2,2,2000248,2002,9,Male,White,"Less than $10,000 (including loss)",None,None,Blank (No Answer),No,LEGITIMATE SKIP,skip,NaN,99,NaN
3,3,2000471,2002,12,Male,Black/African American,"Less than $10,000 (including loss)",Private Health Insurance,Prescriptions,Yes,No,LEGITIMATE SKIP,skip,NaN,99,NaN
4,4,2000630,2002,16,Female,White,"$40,000 to $49,999",Medicare and Group Offered Health Insurance,Prescriptions,Yes,No,LEGITIMATE SKIP,skip,NaN,99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599402,599402,99979928,2019,11,Male,Multiple Races,"$75,000 or more",Group Offered Health Insurance,None,Yes,No,LEGITIMATE SKIP,skip,NaN,99,NaN
599403,599403,99980998,2019,12,Male,White,"$10,000 to $19,999",Group Offered Health Insurance,None,Yes,No,LEGITIMATE SKIP,skip,NaN,99,NaN
599404,599404,99983445,2019,16,Female,White,"$50,000 to $74,999",Group Offered Health Insurance,None,Do not Know,No,LEGITIMATE SKIP,skip,NaN,99,NaN
599405,599405,99985080,2019,16,Male,White,"$30,000 to $39,999",None,None,LEGITIMATE SKIP (PRVHLTIN = 2),No,LEGITIMATE SKIP,skip,NaN,99,NaN


### Final data file: final_data.csv

# Data used for Choropleth Map

In [23]:
import geopandas as gpd

In [24]:
file = 'per.csv'

In [25]:
# Original data for 2014-2015 is in the format of %
df_per = pd.read_csv(file)
df_per.loc[df_per.Year.isin([2014, 2015]), 'sermh_per'] = df_per[df_per.Year.isin([2014, 2015])].sermh_per.div(100).round(5)
df_per.loc[df_per.Year.isin([2014, 2015]), 'anymh_per'] = df_per[df_per.Year.isin([2014, 2015])].anymh_per.div(100).round(5)

In [26]:
# Add column Type; combine sermh_per and anymh_per into column Cases
df_anymh = df_per.copy()
df_anymh = df_anymh.drop(columns = ['sermh_per'])
df_anymh.loc[:,'Type'] = 'Percent of Any Mental Illnesses'
df_anymh = df_anymh.rename(columns = {'anymh_per':'Cases'})

df_sermh = df_per.copy()
df_sermh = df_sermh.drop(columns = ['anymh_per'])
df_sermh.loc[:,'Type'] = 'Percent of Serious Mental Illnesses'
df_sermh = df_sermh.rename(columns = {'sermh_per':'Cases'})

# Combine the df's
df_all_per = pd.concat([df_anymh, df_sermh], ignore_index=True)

In [27]:
# Read csv file for state percentages of population who received services
df_received = pd.read_csv('map_received.csv')
df_received

,State,Cases,Year,Type
0,Alabama,0.1397,2017,Percent of Population Received Services
1,Alaska,0.1302,2017,Percent of Population Received Services
2,Arizona,0.1324,2017,Percent of Population Received Services
3,Arkansas,0.1607,2017,Percent of Population Received Services
4,California,0.1172,2017,Percent of Population Received Services
...,...,...,...,...
148,Virginia,0.1485,2019,Percent of Population Received Services
149,Washington,0.1801,2019,Percent of Population Received Services
150,West Virginia,0.1885,2019,Percent of Population Received Services
151,Wisconsin,0.1926,2019,Percent of Population Received Services


In [28]:
# Combine the dataframes
df_all = pd.concat([df_all_per, df_received], ignore_index = True)
df_all

,State,Cases,Year,Type
0,Alabama,0.212901,2019,Percent of Any Mental Illnesses
1,Alaska,0.214692,2019,Percent of Any Mental Illnesses
2,Arizona,0.200635,2019,Percent of Any Mental Illnesses
3,Arkansas,0.203352,2019,Percent of Any Mental Illnesses
4,California,0.194866,2019,Percent of Any Mental Illnesses
...,...,...,...,...
760,Virginia,0.148500,2019,Percent of Population Received Services
761,Washington,0.180100,2019,Percent of Population Received Services
762,West Virginia,0.188500,2019,Percent of Population Received Services
763,Wisconsin,0.192600,2019,Percent of Population Received Services


In [29]:
# convert df to csv
df_all.to_csv('map.csv', index = False)